<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting Negative Weight Cycles with Priority-Based SPFA
**Author:** Biruk

This Colab notebook demonstrates how to use a priority-queue-based SPFA variant for faster negative cycle detection in graphs.

---

# 🚀 A Faster Way to Detect Negative Weight Cycles

This notebook explores a **heap-based variation** of the Shortest Path Faster Algorithm (SPFA) for detecting negative weight cycles more efficiently. It takes inspiration from:

- 🔍 **Dijkstra’s Algorithm** — for using a **priority queue** to prioritize important nodes.
- ⚡ **SPFA** — an efficient queue-based version of Bellman-Ford that only processes updated nodes.

---

## 📘 Dijkstra’s Algorithm

- Efficient for graphs with **non-negative** weights.
- Uses a **priority queue** to process nodes with the smallest distance first.
- **Limitation:** Cannot handle **negative edge weights**.

➡️ **Takeaway:** The idea of prioritizing nodes using a heap.

---

## 📘 Bellman-Ford Algorithm

- Supports **negative weights**.
- Repeatedly relaxes edges **V - 1** times.
- A further update on the **V-th iteration** means a **negative cycle exists**.

➡️ **Drawback:** Blind relaxation of all edges.

---

## 📘 SPFA (Shortest Path Faster Algorithm)

- Optimized Bellman-Ford using a **queue**.
- Only enqueues a node if its distance improves.
- Can still have poor worst-case performance.

➡️ **Goal:** Improve node selection strategy to converge faster.

---

## 🎯 Better Prioritization Strategy

### Criteria to Prioritize Nodes in Heap:

1. **Indegree:** Fewer incoming edges means fewer relaxations. Low indegree is better.
2. **Outdegree:** More outgoing edges mean more potential relaxations. High outdegree is better.
3. **Outdegree - Indegree:** Measures a node's influence. Larger is better.
4. **Minimum Outgoing Edge Weight:** Smaller weights are more likely to trigger relaxations.
5. **Number of Negative Outgoing Edges:** Risky nodes worth checking early.

# ✅ **Heuristic Used in this Notebook:**

# heap = [(min_edge_weight[source], -out_in[source], source)]
- Primary sort: **Minimum outgoing edge weight**.
- Secondary sort: **Outdegree - Indegree** (negated for max-priority)
# 🧪 Results Summary
- **Complete Graphs:** Custom heap-based SPFA significantly reduced relaxations.
- **Sparse Graphs (e.g. 15 edges/node):** Achieved up to **1/1000th** the relaxations compared to SPFA.

For more detailed explanations and examples, check out this [link](https://medium.com/@birukg500/a-faster-way-to-detect-negative-weight-cycles-56ef93a8af8a). You can also run the algorithms with different inputs to see how they perform. 😆


In [2]:
# Let’s implement and compare these two algorithms below 👇

import random
import heapq
from collections import deque, defaultdict
import time

# === Graph Generation Functions ===

def generate_complete_graph(n, weight_range=(-2, 45)):
    graph = defaultdict(list)
    min_edge_weight = [float('inf')] * n
    out_in = [0] * n

    for u in range(n):
        for v in range(n):
            if u != v:
                weight = random.randint(*weight_range)
                graph[u].append((v, weight))
                out_in[u] += 1
                out_in[v] -= 1
                if weight < min_edge_weight[u]:
                    min_edge_weight[u] = weight

    return graph, min_edge_weight, out_in

def generate_sparse_graph(n, edges_per_node=5, weight_range=(-20, 105)):
    graph = defaultdict(list)
    min_edge_weight = [float('inf')] * n
    out_in = [0] * n

    for u in range(n):
        neighbors = set()
        while len(neighbors) < min(edges_per_node, n - 1):
            v = random.randint(0, n - 1)
            if v != u and v not in neighbors:
                neighbors.add(v)
                weight = random.randint(*weight_range)
                graph[u].append((v, weight))
                out_in[u] += 1
                out_in[v] -= 1
                if weight < min_edge_weight[u]:
                    min_edge_weight[u] = weight

        if not graph[u]:
            v = (u + 1) % n
            weight = random.randint(*weight_range)
            graph[u].append((v, weight))
            out_in[u] += 1
            out_in[v] -= 1
            if weight < min_edge_weight[u]:
                min_edge_weight[u] = weight

    return graph, min_edge_weight, out_in


# === Standard SPFA Algorithm ===

def spfa(graph, source, n):
    dist = [float('inf')] * n
    in_queue = [False] * n
    dist[source] = 0
    q = deque([source])
    in_queue[source] = True

    count = [0] * n
    enqueue_count = 0
    relaxtion_count = 0

    while q:
        u = q.popleft()
        in_queue[u] = False
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                relaxtion_count += 1
                dist[v] = dist[u] + w
                if not in_queue[v]:
                    q.append(v)
                    in_queue[v] = True
                    enqueue_count += 1
                count[v] += 1
                if count[v] > n:
                    print("SPFA: Negative cycle detected.")
                    return None, enqueue_count, relaxtion_count

    return dist, enqueue_count, relaxtion_count


# === Custom SPFA with Priority Queue ===

def custom_spfa_heap(graph, source, n, min_edge_weight, out_in):
    dist = [float('inf')] * n
    in_heap = [False] * n
    dist[source] = 0
    heap = [(min_edge_weight[source], -out_in[source], source)]
    in_heap[source] = True

    count = [0] * n
    enqueue_count = 0
    relaxtion_count = 0

    while heap:
        _, _, u = heapq.heappop(heap)
        in_heap[u] = False

        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                relaxtion_count += 1
                dist[v] = dist[u] + w
                if not in_heap[v]:
                    heapq.heappush(heap, (min_edge_weight[v], -out_in[v], v))
                    in_heap[v] = True
                    enqueue_count += 1
                count[v] += 1
                if count[v] > n:
                    print("Custom SPFA-Heap: Negative cycle detected.")
                    return None, enqueue_count, relaxtion_count

    return dist, enqueue_count, relaxtion_count


# === Run and Compare Both Algorithms ===

n = 2000  # Change size to 2000 or 5000 for more aggressive testing
source = 0

graph, min_weights, imbalance = generate_sparse_graph(n, edges_per_node=15)
# graph, min_weights, imbalance = generate_complete_graph(n)

start = time.time()
spfa_result = spfa(graph, source, n)
spfa_time = time.time() - start

start = time.time()
custom_result = custom_spfa_heap(graph, source, n, min_weights, imbalance)
custom_time = time.time() - start

print("SPFA Result:", spfa_result)
print("Custom SPFA-Heap Result:", custom_result)
print(f"SPFA Time: {spfa_time:.6f} seconds")
print(f"Custom SPFA-Heap Time: {custom_time:.6f} seconds")

SPFA: Negative cycle detected.
Custom SPFA-Heap: Negative cycle detected.
SPFA Result: (None, 1093984, 2143828)
Custom SPFA-Heap Result: (None, 192488, 1561740)
SPFA Time: 2.905058 seconds
Custom SPFA-Heap Time: 0.765837 seconds
